# Relationships

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 1: Create properties

In [ ]:
%%luminesce

-- =============================================================
-- Description:
-- In this query, we create a set of LE identifier properties
-- =============================================================
-- 1. Create new property definition
@@propertyScope = select 'ibor';

@property_definition = select
'Custodian' as [DisplayName],
'LegalEntity' as [Domain],
@@propertyScope as [PropertyScope],
'Custodian' as [PropertyCode],
'Identifier' as [ConstraintStyle],
'system' as [DataTypeScope],
'string' as [DataTypeCode]
union all
values
('CustomLegalEntityId',  'LegalEntity',  @@propertyScope, 'CustomLegalEntityId', 'Identifier', 'system', 'string'),
('Country',  'LegalEntity',  @@propertyScope, 'Country', 'Property', 'system', 'string');

-- 2. Upload new properties to LUSID and print result to console here:
select * from Lusid.Property.Definition.Writer
where ToWrite = @property_definition;


#### Step 2: Upsert instrument properties

In [ ]:
%%luminesce

-- =============================================================
-- Description:
-- In this query, we inline the Legal Entity properties
-- =============================================================
-- 2.  Define custom prop:
@newProperties =
values
('LegalEntity/ibor/Custodian', 'Text', 'Custodian', 'Custodian'),
('LegalEntity/ibor/Country',  'Text', 'Country', 'Country');

-- 4. Write new and old properties into LUSID

@property_definition = select
    Column3 as [DisplayName],
    'LegalEntity' as [Domain],
    'ibor' as [PropertyScope],
    Column3 as [PropertyCode],
    'Property' as [ConstraintStyle],
    'system' as [DataTypeScope],
    case when Column2 == 'Text'
            then 'string'
            else 'number'
      end as [DataTypeCode]
    from @newProperties;

@create_properties =
select * from Lusid.Property.definition.Writer
    where ToWrite = @property_definition;

-- The results of writing the new property definitions can be seen from the query below:
select * from @create_properties;


#### Step 3: Create legal entities

In [ ]:
%%luminesce

-- =============================================================
-- Description:
-- In this query, we create a set of LE identifier and
-- upload values for default and custom properties
-- =============================================================
@le_data =

use Drive.Excel
--file=/luminesce-examples/custodians.xlsx
--worksheet=custodians
enduse;

-- 1. Create LE identifiers and write in built properties to Lusid.LegalEntity
@legalentity_properties =
select custodian_name as DisplayName, custodian_name as Description, lei as LEI
from @le_data;

select * from Lusid.LegalEntity.Writer where ToWrite = @legalentity_properties;

-- 2. Write custom properties values for LE identifiers to Lusid.Property
@inst_properties =
select le.LEI, 'LEI' as EntityIdType, 'LegalEntity' as Domain, 'ibor' as PropertyScope, a.PropertyCode, a.Value, a.
   EntityId
from Lusid.LegalEntity le
inner join (
   select 'Custodian' as PropertyCode, custodian_code as Value, lei as EntityId
   from @le_data
   union
   select 'Country' as PropertyCode, country_code as Value, lei as EntityId
   from @le_data
   ) a
   on le.LEI = a.EntityId;

select * from Lusid.Property.Writer where ToWrite = @inst_properties;



#### Step 4: Create portfolios

In [ ]:
%%luminesce

-- =============================================================
-- Description:
-- 1. In this query, we create a set of Transaction Portfolios
-- =============================================================



@@scope = select 'ibor';

@portfolio_data = use Drive.Excel
--file=/luminesce-examples/custodians.xlsx
--worksheet=portfolios
enduse;

@portfolios_for_upload = select  'Transaction' as PortfolioType,
port_code as PortfolioCode, 
@@scope as PortfolioScope,
port_code as DisplayName,
portfolio_currency as BaseCurrency,
'FirstInFirstOut' as AccountingMethod,
'' as Description, 
#2000-01-01# as Created,
''  as SubHoldingKeys
from @portfolio_data;

select *
from Lusid.Portfolio.Writer
where ToWrite = @portfolios_for_upload;


#### Step 5: Assign lei to portfolio

In [ ]:
%%luminesce

-- ================================================================
-- Description:
-- In this query, we assign Portfolios to a Legal Entity Identifier
-- ================================================================
@@scope = select 'ibor';

-- 1. Create view with LEIs and portfolios joined on custodian name.
@portfolio_data =
use Drive.Excel
--file=/luminesce-examples/custodians.xlsx
--worksheet=portfolios
enduse;

@custodians_data =
use Drive.Excel
--file=/luminesce-examples/custodians.xlsx
--worksheet=custodians
enduse;

@portfolio_custodian_data = select p.port_code, c.lei
from @portfolio_data p
inner join (
   select lei, custodian_code
   from @custodians_data
   ) c
   on p.custodian_code = c.custodian_code;

-- 2. Define a relationship between LEI and Portfolio
@assign_relationships =
select 'LegalEntity' as EntityType, 'LEI' as EntityCode, 'default' as EntityScope, lei as EntityValue, port_code as
   RelatedEntityCode, @@scope as RelatedEntityScope, 'Portfolio' RelatedEntityType, 'LEIToPortfolio' as RelationshipCode,
   @@scope as RelationshipScope
from @portfolio_custodian_data;

-- 3. Write the relationship to Lusid.Relationship.Writer and print results to console
select *
from Lusid.Relationship.Writer
where ToWrite = @assign_relationships;



#### Step 6: Create relationship definition

In [ ]:
%%luminesce

-- ====================================================================
-- Description:
-- 1. In this query, we create a relationship entity to 
-- capture the mapping between Portfolios and Legal Entity Identifiers
-- ====================================================================

@port_custodian_relationship = select 'LEIToPortfolio' as Code,
'LEIToPortfolio'as DisplayName,
'LEIToPortfolio' as OutwardDescription,
'PortfolioToLEI' as InwardDescription,
'default' as Scope,
'LegalEntity' as SourceEntityType,
'Portfolio' as TargetEntityType;

select * from Lusid.Relationship.Definition.Writer
where ToWrite = @port_custodian_relationship;


#### Step 7: Call back custodians

In [ ]:
%%luminesce

-- =============================================================
-- Description:
-- In this query, we create a report of Portfolios to
-- Legal Entity Identifiers
-- =============================================================
-- 1. Collect the Legal Entity Identifiers
@lei_data =
use Drive.Excel
--file=/luminesce-examples/custodians.xlsx
--worksheet=custodians
enduse;

@leis =
select 'LegalEntity' as EntityType, 'LEI' as EntityCode, 'default' as EntityScope, lei as EntityValue
from @lei_data;

-- 2. Check which Portfolios in Lusid have the same custodians as these Legal Entities
@relationships =
select RelationshipCode, EntityValue as 'LegalEntity', RelatedEntityCode as 'Portfolio'
from Lusid.Relationship
where ToLookUp = @leis;

-- 3. Generate report of Legal Entities to Portfolios
@ids =
select EntityValue
from @leis;

@report =

select p.EntityId, p.Value as Custodian, r.RelationshipCode, r.Portfolio
from Lusid.Property p
inner join (
   select *
   from @relationships
   ) r
   on r.LegalEntity = p.EntityId
where p.propertycode = 'Custodian'
   and p.propertyscope = 'ibor'
   and p.domain = 'LegalEntity'
   and p.EntityId in @ids
   and p.EntityIdType = 'LEI'
   and p.EntityScope = 'default';

select *
from @report;

